In [11]:
from collections import Counter 
import requests
from bs4 import BeautifulSoup as soup
import json
import json_lines

commentator_link = 'https://www.blueletterbible.org/commentaries/edersheim_alfred/'


In [12]:
r = requests.get(commentator_link)


# https://stackoverflow.com/questions/44203397/python-requests-get-returns-improperly-decoded-text-instead-of-utf-8
r.encoding = r.apparent_encoding

link = soup(r.text)

### 1. Getting the links to the works of the commentator

In [23]:
['asd','dsa'][None:]

['asd', 'dsa']

In [27]:
def commentatorpage_2_links(commentator_link, document_nm_root = '', drop_first=None, drop_last=None):
    """
    Get the links of the commentators work with their respective 
    """
    root_link = commentator_link.split('.org')[0] + '.org'
    r = requests.get(commentator_link)
    r.encoding = r.apparent_encoding
    link = soup(r.text)
    list_of_elements = link.findAll('li',{'class':'articleLI'})
    
    authors = ' '.join([name.capitalize() for name in commentator_link.split('/')[-2].split('_')])
    
    #  get link and document name
    chapter_links = [{
                      'url': root_link+element.a['href'], 
                      'verse_reference':'',
                      'document_nm':document_nm_root+' - '+element.a.text,
                      'authors':authors,
                     }
                     for element in list_of_elements]
    chapter_links = chapter_links[drop_first:]
    chapter_links = chapter_links[:-drop_last]
    
    return chapter_links

commentatorpage_2_links('https://www.blueletterbible.org/commentaries/edersheim_alfred/',
                        document_nm_root = 'The New Schaff-Herzog Encyclopedia of Religious Knowledge',
                        drop_last=3
                       )

[{'url': 'https://www.blueletterbible.org/Comm/edersheim_alfred/sketches/sketches00.cfm',
  'verse_reference': '',
  'document_nm': 'The New Schaff-Herzog Encyclopedia of Religious Knowledge - Preface',
  'authors': 'Edersheim Alfred'},
 {'url': 'https://www.blueletterbible.org/Comm/edersheim_alfred/sketches/sketches01.cfm',
  'verse_reference': '',
  'document_nm': 'The New Schaff-Herzog Encyclopedia of Religious Knowledge - Chapter 1. Palestine Eighteen Centuries Ago',
  'authors': 'Edersheim Alfred'},
 {'url': 'https://www.blueletterbible.org/Comm/edersheim_alfred/sketches/sketches02.cfm',
  'verse_reference': '',
  'document_nm': 'The New Schaff-Herzog Encyclopedia of Religious Knowledge - Chapter 2. Jews and Gentiles in “The Land”',
  'authors': 'Edersheim Alfred'},
 {'url': 'https://www.blueletterbible.org/Comm/edersheim_alfred/sketches/sketches03.cfm',
  'verse_reference': '',
  'document_nm': 'The New Schaff-Herzog Encyclopedia of Religious Knowledge - Chapter 3. In Galilee at 

### 2. Getting the paragraph of the chapter link

In [14]:
chapter_link = 'https://www.blueletterbible.org/Comm/edersheim_alfred/sketches/sketches00.cfm'

def remove_footnote_link(string):
    try:
        splitted=string.split('\xa0')
        a=splitted[0]
        b='x'
        if len(splitted)>1:
            b=splitted[1]
        return ''.join([a,b[1:]])
    except:
        print('remove_footnote_link - failed to remove ascii'.upper())
        print(string)
        print(splitted)
        print(a)
        print(b)
        raise
        return string

def chapter_to_list_of_paragraphs( chapter_link ):
    """
    
    
    args:
    ----
        chapter_link: (str) this contains the link to the commentators work
    return:
    ------
        list_of_paragraphs: (list) list of paragraph string
    """
    r = requests.get(chapter_link)
    # https://stackoverflow.com/questions/44203397/python-requests-get-returns-improperly-decoded-text-instead-of-utf-8
    r.encoding = r.apparent_encoding
    link = soup(r.text)
    
    # remove footnotes
    link = soup(str(link.find("div", {"id":"commData"})).split("<hr")[0])
    
    paragraph_elements = link.find("div", {"id":"commData"}).findAll("p")
    list_of_paragraphs = [paragraph_element.text.replace('\n','').strip() 
                          for paragraph_element in paragraph_elements
                          if len(paragraph_element.text.replace('\n','').strip().split(' ') )> 7
                         ]
    
    # remove footnote links 
    list_of_paragraphs = [remove_footnote_link(paragraph)
                          for paragraph in list_of_paragraphs
                         ]
    
    return list_of_paragraphs#, paragraph_elements, link

list_of_paragraphs = chapter_to_list_of_paragraphs( "https://www.blueletterbible.org/Comm/ice_thomas/Mat24-25/Mat24-25_Part01.cfm" )

However, there are paragraphs that we may not need.
1. Quotes - we may want to attach them to the previous paragraph.
2. Indented paragraphs - similar to quotes
<font color=green>
3. Footnotes - We may want to remove them altogether. The superscript link to the footnotes will write "\xa02"  
    

A link with quotes and footnotes is:  
https://www.blueletterbible.org/Comm/ice_thomas/Mat24-25/Mat24-25_Part01.cfm

In [15]:
test_str = 'The Olivet Discourse, delivered shortly before Jesus’ crucifixion, is the most important single passage of prophecy in all the Bible. It is significant because it came from Jesus Himself immediately after He was rejected by His own people and because it provides the master outline of end-time events.—Dr. Tim LaHaye\xa01'
test_str.split('\xa0')[0]

'The Olivet Discourse, delivered shortly before Jesus’ crucifixion, is the most important single passage of prophecy in all the Bible. It is significant because it came from Jesus Himself immediately after He was rejected by His own people and because it provides the master outline of end-time events.—Dr. Tim LaHaye'

To scrape BLB, we need `commentator_link, document_nm_root`, `drop_first`, `drop_last` which are the links and the wider book name, slicing indices (to be manually keyed in) respectively. 

### The scraping logic

In [16]:
"""
Scrape selected authors, their pages
"""
from collections import Counter 
import requests
from bs4 import BeautifulSoup as soup
import json
import json_lines

def commentatorpage_2_links(commentator_link, document_nm_root = '', drop_first=None, drop_last=None):
    """
    Get the links of the commentators work with their respective 
    """
    root_link = commentator_link.split('.org')[0] + '.org'
    r = requests.get(commentator_link)
    r.encoding = r.apparent_encoding
    link = soup(r.text)
    list_of_elements = link.findAll('li',{'class':'articleLI'})
    
    authors = ' '.join([name.capitalize() for name in commentator_link.split('/')[-2].split('_')])
    
    #  get link and document name
    chapter_links = [{
                      'url': root_link+element.a['href'], 
                      'verse_reference':'',
                      'document_nm':document_nm_root+' - '+element.a.text,
                      'authors':authors,
                     }
                     for element in list_of_elements]
    chapter_links = chapter_links[drop_first:]
    chapter_links = chapter_links[:-drop_last]
    
    return chapter_links


def remove_footnote_link(string):
    try:
        splitted=string.split('\xa0')
        a=splitted[0]
        b='x'
        if len(splitted)>1:
            b=splitted[1]
        return ''.join([a,b[1:]])
    except:
        print('remove_footnote_link - failed to remove ascii'.upper())
        print(string)
        print(splitted)
        print(a)
        print(b)
        raise
        return string

def chapter_to_list_of_paragraphs( chapter_link ):
    """
    
    
    args:
    ----
        chapter_link: (str) this contains the link to the commentators work
    return:
    ------
        list_of_paragraphs: (list) list of paragraph string
    """
    r = requests.get(chapter_link)
    # https://stackoverflow.com/questions/44203397/python-requests-get-returns-improperly-decoded-text-instead-of-utf-8
    r.encoding = r.apparent_encoding
    link = soup(r.text)
    
    # remove footnotes
    link = soup(str(link.find("div", {"id":"commData"})).split("<hr")[0])
    
    paragraph_elements = link.find("div", {"id":"commData"}).findAll("p")
    list_of_paragraphs = [paragraph_element.text.replace('\n','').strip() 
                          for paragraph_element in paragraph_elements
                          if len(paragraph_element.text.replace('\n','').strip().split(' ') )> 7
                         ]
    
    # remove footnote links 
    list_of_paragraphs = [remove_footnote_link(paragraph)
                          for paragraph in list_of_paragraphs
                         ]
    
    return list_of_paragraphs#, paragraph_elements, link


if __name__=="__main__":
    
    blb_catalogue = [
        ()
        
        
    ]
    
    for commentator_link, document_nm_root, drop_first, drop_last in blb_catalogue:

        # 1. get chapter links, 
        #    which is a list of dictionaries,
        #    detailing url, verse_ref, document_nm, authors
        chapter_links = commentatorpage_2_links(commentator_link, document_nm_root = '', drop_first=None, drop_last=None)


        for commentator_chapter_link in chapter_links:

            # 2. for each commentator_chapter_link, 
            #    get a list of paragraphs from chapter
            list_of_paragraphs = chapter_to_list_of_paragraphs( commentator_chapter_link )

            for paragraph in list_of_paragraphs:

                # 3. save in jsonl format and save to kb

                json_row_to_save = commentator_chapter_link.copy()
                json_row_to_save['paragraph'] = paragraph

                with open("data/blueletterbible.jsonl", 'a') as outfile:
                    json.dump(json_row_to_save, outfile)
                    outfile.write('\n')
            

['The Olivet Discourse, delivered shortly before Jesus’ crucifixion, is the most important single passage of prophecy in all the Bible. It is significant because it came from Jesus Himself immediately after He was rejected by His own people and because it provides the master outline of end-time events.—Dr. Tim LaHaye',
 'Bible prophecy. The Olivet Discourse is made up of our Lord’s teaching on Bible prophecy that is found in Matthew 24-25, Mark 13 and Luke 21. Since one’s interpretation of the Olivet Discourse greatly impacts whether they are a premillennialist or anti-millennialist, futurist or preterist, or pretribulationists or posttribulationist, I will be attempting an extensive interpretation of Matthew 24-25.',
 'The setting for the Olivet Discourse, at least for Matthew’s account, is found in preceding events leading up to Matthew 24. Christ had presented Himself to the nation as their Messiah, but they rejected Him. No only did the people reject Him, but their rulers did as we

### Troubleshoot

In [30]:
chapter_links

NameError: name 'chapter_links' is not defined